# Vibe Matcher — 

**Why AI at Nexora?**

Nexora operates at the intersection of creativity and customer experience — AI enables highly-personalized product discovery by turning free-text 'vibe' queries into dense representations, matching customers to items that reflect their mood and context. This increases engagement, reduces choice friction, and enables contextual merchandising like festival, evening, or cozy home looks that drives conversion while preserving brand aesthetics.

This notebook demonstrates a compact prototype: dataset creation, embedding (simulated), similarity search, evaluation, latency measurement, and reflections for production improvements.


In [11]:
# Dataset
products = [
  {
    "id": 1,
    "name": "Boho Dress",
    "desc": "Flowy linen dress, earthy tones, layered for festival bohemian vibes.",
    "tags": [
      "boho",
      "festival",
      "flowy"
    ]
  },
  {
    "id": 2,
    "name": "Urban Moto Jacket",
    "desc": "Cropped black faux-leather jacket with metallic zips \u2014 energetic urban chic.",
    "tags": [
      "urban",
      "edgy",
      "chic"
    ]
  },
  {
    "id": 3,
    "name": "Cozy Knit Sweater",
    "desc": "Chunky knit, oversized, warm and comfy for relaxed cozy days.",
    "tags": [
      "cozy",
      "casual",
      "warm"
    ]
  },
  {
    "id": 4,
    "name": "Minimalist Slip Dress",
    "desc": "Satin slip dress, clean lines \u2014 modern minimalist evening wear.",
    "tags": [
      "minimalist",
      "elegant",
      "evening"
    ]
  },
  {
    "id": 5,
    "name": "Sporty Mesh Sneakers",
    "desc": "Lightweight mesh trainers with vibrant accents for active streetwear looks.",
    "tags": [
      "sporty",
      "streetwear",
      "active"
    ]
  },
  {
    "id": 6,
    "name": "Pastel Cardigan",
    "desc": "Soft pastel cardigan \u2014 cute, gentle, and cozy cottage-core vibes.",
    "tags": [
      "cottagecore",
      "cozy",
      "pastel"
    ]
  },
  {
    "id": 7,
    "name": "Denim Wide-Leg Jeans",
    "desc": "High-waisted wide-leg denim, relaxed and retro-inspired urban look.",
    "tags": [
      "retro",
      "urban",
      "casual"
    ]
  }
]
import pandas as pd
df = pd.DataFrame(products)
df

,id,name,desc,tags
0,1,Boho Dress,"Flowy linen dress, earthy tones, layered for f...","[boho, festival, flowy]"
1,2,Urban Moto Jacket,Cropped black faux-leather jacket with metalli...,"[urban, edgy, chic]"
2,3,Cozy Knit Sweater,"Chunky knit, oversized, warm and comfy for rel...","[cozy, casual, warm]"
3,4,Minimalist Slip Dress,"Satin slip dress, clean lines — modern minimal...","[minimalist, elegant, evening]"
4,5,Sporty Mesh Sneakers,Lightweight mesh trainers with vibrant accents...,"[sporty, streetwear, active]"
5,6,Pastel Cardigan,"Soft pastel cardigan — cute, gentle, and cozy ...","[cottagecore, cozy, pastel]"
6,7,Denim Wide-Leg Jeans,"High-waisted wide-leg denim, relaxed and retro...","[retro, urban, casual]"


In [13]:

# Build TF-IDF based embeddings and VibeMatcher class (simulated embeddings for demo)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
class VibeMatcher:
    def __init__(self, product_df, text_field="desc"):
        self.df = product_df.reset_index(drop=True).copy()
        self.text_field = text_field
        self.vectorizer = TfidfVectorizer(ngram_range=(1,2), stop_words='english')
        self.emb_matrix = self.vectorizer.fit_transform(self.df[self.text_field].astype(str).tolist())
    def embed_query(self, query_text):
        return self.vectorizer.transform([query_text])
    def query(self, query_text, top_k=3, score_threshold=0.7):
        q_vec = self.embed_query(query_text)
        sims = cosine_similarity(q_vec, self.emb_matrix).flatten()
        ranked_idx = np.argsort(-sims)
        results = []
        for idx in ranked_idx[:top_k]:
            results.append({
                "id": int(self.df.loc[idx, "id"]),
                "name": self.df.loc[idx, "name"],
                "desc": self.df.loc[idx, "desc"],
                "tags": self.df.loc[idx, "tags"],
                "score": float(sims[idx])
            })
        best_score = float(sims[ranked_idx[0]])
        fallback = None
        if best_score < score_threshold:
            fallback = {"message":"No strong matches found. Try broader vibes.","best_score":best_score}
        return {"query": query_text, "results": results, "best_score": best_score, "fallback": fallback}


In [15]:
# queries and results
sample_queries = ["energetic urban chic", "soft cozy cottage-core", "minimalist evening elegance"]
matcher = VibeMatcher(df)
for q in sample_queries:
    out = matcher.query(q)
    print(q, out['best_score'])
    for r in out['results']:
        print(' -', r['name'], r['score'])
    print()

energetic urban chic 0.5008971504441561
 - Urban Moto Jacket 0.5008971504441561
 - Denim Wide-Leg Jeans 0.07422562019107092
 - Boho Dress 0.0

soft cozy cottage-core 0.5893700051738489
 - Pastel Cardigan 0.5893700051738489
 - Cozy Knit Sweater 0.0761861338059701
 - Boho Dress 0.0

minimalist evening elegance 0.42397955644411445
 - Minimalist Slip Dress 0.42397955644411445
 - Boho Dress 0.0
 - Urban Moto Jacket 0.0



**By:** Pallavi Gorakshanath Akolkar  
B.E. IT | AI & Web Development Enthusiast
